# Getting Started with Object Detection
An introduction to the Velour evaluation service.

In [7]:
from velour import Client
from velour.coretypes import (
    Dataset,
    Model,
    Datum,
    Annotation,
    GroundTruth, 
    Prediction,
    Label,
)
from velour.schemas import BoundingBox
from velour.enums import TaskType

Connect to the API

In [8]:
client = Client("http://localhost:8000")

Succesfully connected to http://localhost:8000/.


Create `Dataset` and `Model` using the `create` classmethod. These objects will act as the link to the Backend.

In [9]:
dataset = Dataset.create(client, name="myDataset")
model = Model.create(client, name="myModel")

ClientException: Dataset with name `myDataset` already exists.

Create a `Datum`

In [ ]:
datum = Datum(uid="myDatum1")

Create a few `BoundingBox` for use later.

In [ ]:
bbox1 = BoundingBox.from_extrema(0, 10, 0, 10)
bbox2 = BoundingBox.from_extrema(2, 12, 3, 13)

Create an `Annotation` for a `GroundTruth`

In [ ]:
annotation = Annotation(
    task_type=TaskType.DETECTION,
    labels=[
        Label(key="k1", value="v1"),
        Label(key="k2", value="v2"),
    ],
    bounding_box=bbox1,
)

NameError: name 'bbox1' is not defined

Create an `Annotation` for a `Prediction`

In [ ]:
annotation_with_scores = Annotation(
    task_type=TaskType.DETECTION,
    labels=[
        Label(key="k1", value="v1", score=0.7),
        Label(key="k1", value="v2", score=0.3),
        Label(key="k2", value="v1", score=0.1),
        Label(key="k2", value="v2", score=0.9),
    ]
    bounding_box=bbox2,
)

`velour.coretypes` also defines `GroundTruth` and `Prediction`. These are closely related objects with the only difference being the backend target.

In [ ]:
groundtruth = GroundTruth(datum, annotations=[annotation])
prediction = Prediction(datum, annotations=[annotation_with_scores])

We now have everything required to upload to velour! 

In [ ]:
dataset.add_groundtruth(groundtruth)
model.add_prediction(prediction)

Before we can run an evaluation we need to finalize our `Dataset` and `Model`. This is a requirement of Velour so that repeat evaluations over a Dataset-Model pairing are guaranteed to return the same result.

In [ ]:
dataset.finalize()

`Model` finalization is a little different than `Dataset` as it is locking predictions with respect to the dataset they operated over.

In [ ]:
model.finalize_inferences(dataset)

We now have a dataset and model representation on the backend that we can evaluate!

In [ ]:
evaluation = model.evaluate_classification(dataset)
evaluation.wait_for_completion()

We currently support `classification`, `object-detection`, and `semantic-segmentation` evaluations.

In [ ]:
print(evaluation.results)